In [1]:
import logging

from netgen.webgui import Draw
from ngsolve import TaskManager
from ngsolve.webgui import Draw as DrawNG

import ossdbs
from ossdbs.utils.settings import Settings


ossdbs.set_logger(logging.INFO)

In [2]:
settings = {
    "BrainRegion": {
        "Center": {"x[mm]":0, "y[mm]": 0, "z[mm]": 0},
        "Dimension": {"x[mm]": 60, "y[mm]": 60, "z[mm]": 80},
        "Shape": "Ellipsoid"
    },
    "Electrodes": [
        {
            "Name": "BostonScientificVercise",
            "CustomParameters": {
                "tip_length": 1.1,
                "contact_length": 1.5,
                "contact_spacing": 0.5,
                "lead_diameter": 1.3,
                "total_length": 450.0,
            },
            "Rotation[Degrees]":  0,
            "Direction": {"x[mm]": 0, "y[mm]": 0, "z[mm]": 1},
            "TipPosition": {"x[mm]": 0, "y[mm]": 0, "z[mm]": 0},
            "Contacts": [
                       {
                    "Contact_ID": 1,
                    "Active": True,
                    #"Current[A]": 0.0,
                    "Voltage[V]": 0.0,
                    "Floating": False,
                  
                },
                {
                    "Contact_ID": 2,
                    "Active": True,
                    #"Current[A]": 0.012,
                    "Voltage[V]": 3.7,
                    "Floating": False,
                    
                },
                {
                    "Contact_ID": 3,
                    "Active": False,
                    #"Current[A]": -0.004,
                    "Voltage[V]":0.0,
                    "Floating": True,
                    
                },
                {
                    "Contact_ID": 4,
                    "Active": False,
                    #"Current[A]": -0.008,
                    "Voltage[V]": 0.0,
                    "Floating": True,
                    
                }
            
        
             
            ],
            "EncapsulationLayer": {
                "Thickness[mm]": 0.1,
                "Material": "Gray matter",
                "DielectricModel": "ColeCole4",
                "MaxMeshSize": 0.5,
            },
        }
    ],
    "MaterialDistribution": {
        "MRIPath": "/Users/shukladas/OSS-DBSv2/examples/ConvergenceStudy/PAM/segmask.nii.gz",
        "MRIMapping": {"Unknown": 0,"Gray matter":1, "White matter": 2, "CSF": 3,"Blood": 4,},
        "DiffusionTensorActive": False,
        "DTIPath": "",
    },
        
   "DielectricModel": {
       "Type": "ColeCole4",
    "CustomParameters": None
       
   },
    "Mesh": {
        "LoadMesh": False,
        "MeshElementOrder": 2,
        "MeshingHypothesis": {"Type": "Default", "MaxMeshSize": 4.0},
        "MeshSize": {"Edges": {}, "Faces": {}, "Volumes": {}},
        "SaveMesh":False,
        "AdaptiveMeshRefinement": {
            "Active": True,
           "MaxIterations": 5,
           "ErrorTolerance": 0.1
        },
    },
    "FEMOrder": 2,
    "EQSMode": True,
    "StimulationSignal": {
        "Type": "Rectangle",
        "Frequency[Hz]": 130.0,
        "PulseWidth[us]": 30.0,
        "PulseTopWidth[us]": 0.0,
        "CounterPulseWidth[us]": 0.0,
        "InterPulseWidth[us]": 0.0,
        "SpectrumMode": "Harmonics",
        "CounterAmplitude": 1.0,
        "CutoffFrequency": 10000.0,
        "CurrentControlled": False
    },
    "Solver": {
        "Type": "CG",
        "Preconditioner": "local",
        "PreconditionerKwargs": {},
        "MaximumSteps": 5000,
        "Precision": 1e-10,
    },
    
    "PointModel": {
        "Pathway": {
            # Pathway gives a prepared neuron array, so when you have this, no need for lattice/voxel lattice
            "Active": False, 
            "FileName": "/Users/shukladas/OSS-DBSv2/examples/ConvergenceStudy/PAM/Allocated_axons.h5",
            "ExportField": True},
        "Lattice": {
            # Even distribution of neuron(axon) points in space
            "Active": False,
            "Center": {"x[mm]": 0, "y[mm]": 0, "z[mm]": 0},
            "Shape": {"x": 10, "y": 10, "z": 10},
            "Direction": {"x[mm]": 0, "y[mm]": 0, "z[mm]": 1},
            "PointDistance[mm]": 0.1,
        },
        "VoxelLattice": {
            # Biologiclally realistic distribution of neuron(axon) points following anatomy
            "Active": True,
            "Shape": {"x": 17, "y": 17, "z": 17} # Creates 17*17*17 = 4913 axon points,
        },
    },
    "PathwayFile": "",
    "OutputPath":"/Users/shukladas/OSS-DBSv2/Results/",
    "ComputeImpedance": True,
    "ActivationThresholdVTA[V-per-m]": 200,
    "ExportVTK": True,
    "ExportElectrode": True,
    "OutOfCore": False,
}


In [3]:
settings = Settings(settings).complete_settings()
mri_image, dti_image = ossdbs.load_images(settings)
electrodes = ossdbs.generate_electrodes(settings)
Draw(electrodes[0].geometry)

INFO:ossdbs.api:Load MRI image
INFO:ossdbs.api:Generate electrode geometries
INFO:ossdbs.electrodes.electrode_model_template:Export electrode as Netgen and VTK file


WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'ngsolve_version': 'Netgen x.x', 'mesh_dim': 3…

BaseWebGuiScene

In [4]:
region_parameters = settings["BrainRegion"]
brain_region = ossdbs.create_bounding_box(settings["BrainRegion"])
shape = settings["BrainRegion"]["Shape"]

brain = ossdbs.BrainGeometry(shape, brain_region)

In [5]:
geometry = ossdbs.ModelGeometry(brain, electrodes)
Draw(geometry.geometry.shape)

INFO:ossdbs.electrodes.electrode_model_template:Boundary names updated


WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'ngsolve_version': 'Netgen x.x', 'mesh_dim': 3…

BaseWebGuiScene

In [6]:
mesh = ossdbs.generate_mesh(settings)

INFO:ossdbs.api:Generate electrode geometries
INFO:ossdbs.electrodes.electrode_model_template:Export electrode as Netgen and VTK file
INFO:ossdbs.electrodes.electrode_model_template:Boundary names updated
INFO:ossdbs.api:Set values on contacts and encapsulation layers
INFO:ossdbs.api:Updating encapsulation layer properties


In [7]:
Draw(mesh.ngsolvemesh)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

BaseWebGuiScene

In [8]:
ossdbs.set_contact_and_encapsulation_layer_properties(settings, geometry)
ossdbs.generate_point_models(settings)

dielectric_model = ossdbs.prepare_dielectric_properties(settings)

materials = settings["MaterialDistribution"]["MRIMapping"]
conductivity = ossdbs.ConductivityCF(
    mri_image,
    brain_region,
    dielectric_model,
    materials,
    geometry.encapsulation_layers,
    complex_data=settings["EQSMode"],
)

solver = ossdbs.prepare_solver(settings)

frequency_domain_signal = ossdbs.prepare_stimulation_signal(settings)

INFO:ossdbs.api:Set values on contacts and encapsulation layers
INFO:ossdbs.api:Updating encapsulation layer properties
INFO:ossdbs.api:from voxel lattice
INFO:ossdbs.api:Prepare dielectric model
INFO:ossdbs.api:Preparing solver


In [9]:
with TaskManager():
    volume_conductor = ossdbs.prepare_volume_conductor_model(
        settings, geometry, conductivity, solver
    )
    ossdbs.run_volume_conductor_model(
        settings, volume_conductor, frequency_domain_signal
    )

INFO:ossdbs.api:Generate volume conductor model
INFO:ossdbs.api:Output path set to: /Users/shukladas/OSS-DBSv2/Results/
INFO:ossdbs.api:Run volume conductor model
INFO:ossdbs.api:Will compute impedance at each frequency
INFO:ossdbs.api:Will export solution to VTK
INFO:ossdbs.api:from voxel lattice
INFO:ossdbs.fem.volume_conductor.volume_conductor_model:Set export frequency to 4940.0
INFO:ossdbs.fem.volume_conductor.volume_conductor_model:Number of elements before material refinement:59143
INFO:ossdbs.fem.volume_conductor.volume_conductor_model:Number of elements after material refinement:59143
INFO:ossdbs.fem.volume_conductor.volume_conductor_model:Computing at frequency: 0.0
INFO:ossdbs.fem.volume_conductor.volume_conductor_model:Number of elements before refinement:59143
INFO:ossdbs.fem.volume_conductor.volume_conductor_model:Adaptive refinement step 1, error 0.148%.
INFO:ossdbs.fem.volume_conductor.volume_conductor_model:Adaptive refinement step 2, error 0.005%.
INFO:ossdbs.fem.volu

INFO:ossdbs.fem.volume_conductor.volume_conductor_model:Exporting electric field at frequency 4940.0Hz.
INFO:ossdbs.fem.volume_conductor.volume_conductor_model:Computing at frequency: 5070.0
INFO:ossdbs.fem.volume_conductor.volume_conductor_model:Copy solution to point models
INFO:ossdbs.fem.volume_conductor.volume_conductor_model:Computing at frequency: 5200.0
INFO:ossdbs.fem.volume_conductor.volume_conductor_model:Copy solution to point models
INFO:ossdbs.fem.volume_conductor.volume_conductor_model:Computing at frequency: 5330.0
INFO:ossdbs.fem.volume_conductor.volume_conductor_model:Copy solution to point models
INFO:ossdbs.fem.volume_conductor.volume_conductor_model:Computing at frequency: 5460.0
INFO:ossdbs.fem.volume_conductor.volume_conductor_model:Copy solution to point models
INFO:ossdbs.fem.volume_conductor.volume_conductor_model:Computing at frequency: 5590.0
INFO:ossdbs.fem.volume_conductor.volume_conductor_model:Copy solution to point models
INFO:ossdbs.fem.volume_conducto

In [10]:
DrawNG(volume_conductor.potential)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {'Complex': {'phase': 0.0, 'sp…

BaseWebGuiScene